In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mtb
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
cd

C:\Users\37789


In [4]:
df = pd.read_csv("Final1.csv")

In [5]:
df

,Airline,Source,Destination,Class,Date,Stopover_1_Time,Stopover_1_Airport,Stopover_2_Time,Stopover_2_Airport,Stopover_3_Time,Stopover_3_Airport,Total_Stopover_Time,Operated,price in CAD,days_left,travel_duration_hours,num_stopovers,departure_hour,arrival_hour,day_of_week
0,Etihad Airways,Toronto Pearson Intl,Bengaluru Intl,Economy,6/2/2024,130,Abu Dhabi Zayed Intl,0,NaN,NaN,NaN,130,NaN,2340,1,28.916667,1,22,3,6
1,Delta,Toronto Pearson Intl,Bengaluru Intl,Economy,6/2/2024,1335,Amsterdam Schiphol,0,NaN,NaN,NaN,1335,KLM,1347,1,48.583333,1,22,23,6
2,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,Economy,6/2/2024,125,Frankfurt am Main,295,Mumbai Chhatrapati Shivaji Intl,NaN,NaN,420,Air Canada,1934,1,34.416667,2,21,7,6
3,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,Economy,6/2/2024,115,Frankfurt am Main,265,Hyderabad Rajiv Gandhi Intl,NaN,NaN,380,Air Canada,2291,1,33.500000,2,21,6,6
4,Multiple Airlines,Toronto Pearson Intl,Bengaluru Intl,Economy,6/2/2024,85,London Heathrow,90,New Delhi Indira Gandhi Intl,NaN,NaN,175,NaN,2661,1,30.750000,2,23,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73799,American Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,Business,7/21/2024,364,Dallas/Fort Worth,0,NaN,NaN,NaN,364,NaN,4032,50,18.400000,1,22,16,6
73800,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,Business,7/21/2024,200,Newark,85,Chicago O'Hare Intl,NaN,NaN,285,Republic Airways DBA United Express,4004,50,17.866667,2,21,14,6
73801,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,Business,7/21/2024,139,Newark,146,Chicago O'Hare Intl,NaN,NaN,285,Republic Airways DBA United Express,4004,50,17.866667,2,21,14,6
73802,United Airlines,Sao Paulo Guarulhos Intl,Toronto Pearson Intl,Business,7/21/2024,100,Houston George Bush Intcntl,172,Chicago O'Hare Intl,NaN,NaN,272,NaN,4004,50,17.883333,2,21,15,6


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73804 entries, 0 to 73803
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Airline                73804 non-null  object 
 1   Source                 73804 non-null  object 
 2   Destination            73804 non-null  object 
 3   Class                  73804 non-null  object 
 4   Date                   73804 non-null  object 
 5   Stopover_1_Time        73804 non-null  int64  
 6   Stopover_1_Airport     73309 non-null  object 
 7   Stopover_2_Time        73804 non-null  int64  
 8   Stopover_2_Airport     50938 non-null  object 
 9   Stopover_3_Time        8150 non-null   object 
 10  Stopover_3_Airport     8150 non-null   object 
 11  Total_Stopover_Time    73804 non-null  int64  
 12  Operated               33505 non-null  object 
 13  price in CAD           73804 non-null  int64  
 14  days_left              73804 non-null  int64  
 15  tr

In [7]:
df.select_dtypes(include=["object"]).columns

Index(['Airline', 'Source', 'Destination', 'Class', 'Date',
       'Stopover_1_Airport', 'Stopover_2_Airport', 'Stopover_3_Time',
       'Stopover_3_Airport', 'Operated'],
      dtype='object')

In [8]:
df.shape

(73804, 20)

In [9]:
df.isnull().sum()

Airline                      0
Source                       0
Destination                  0
Class                        0
Date                         0
Stopover_1_Time              0
Stopover_1_Airport         495
Stopover_2_Time              0
Stopover_2_Airport       22866
Stopover_3_Time          65654
Stopover_3_Airport       65654
Total_Stopover_Time          0
Operated                 40299
price in CAD                 0
days_left                    0
travel_duration_hours        0
num_stopovers                0
departure_hour               0
arrival_hour                 0
day_of_week                  0
dtype: int64

In [10]:
df = df.drop(columns=['Stopover_3_Time', 'Stopover_3_Airport', 'Operated']) # Many null values

In [11]:
df = df.drop(columns=['Stopover_1_Time', 'Stopover_1_Airport', 'Stopover_2_Time', 'Stopover_2_Airport'])
# Not necessary for our model 

In [12]:
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

# Split the date into separate day, month, and year columns
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Drop the original 'Date' column if no longer needed
df = df.drop(columns=['Date'])

In [13]:
df["Class"].unique()

array(['Economy', 'Business', 'Premium Economy', 'First'], dtype=object)

In [14]:
# Initialize the OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the 'Class' column
encoded_array = encoder.fit_transform(df[['Class']]).toarray()

# Convert the array back to a DataFrame with column names
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['Class']))

# Concatenate the original DataFrame with the encoded DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original 'Class' column
df.drop(columns=['Class'], inplace=True)

In [15]:
label_encoder = LabelEncoder()
df['Airline'] = label_encoder.fit_transform(df['Airline'])
df['Source'] = label_encoder.fit_transform(df['Source'])
df['Destination'] = label_encoder.fit_transform(df['Destination'])

In [16]:
y = df['price in CAD']
X = df.drop(columns=['price in CAD'])

In [17]:
X = sm.add_constant(X)

In [18]:
model = sm.OLS(y, X).fit()

In [19]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     4428.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:22:20   Log-Likelihood:            -6.5265e+05
No. Observations:               73804   AIC:                         1.305e+06
Df Residuals:                   73789   BIC:                         1.305e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Airline                   4.07

In [20]:
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)
C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                  Feature       VIF
0                 Airline  1.130902
1                  Source  2.401026
2             Destination  3.372326
3     Total_Stopover_Time  2.029695
4               days_left       inf
5   travel_duration_hours  5.365978
6           num_stopovers  1.456109
7          departure_hour  1.158899
8            arrival_hour  1.231980
9             day_of_week  1.062061
10                    Day       inf
11                  Month       inf
12                   Year  0.000000
13         Class_Business       inf
14          Class_Economy       inf
15            Class_First       inf
16  Class_Premium Economy       inf


In [21]:
X = X.drop(columns=['Day'])
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     4428.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:22:54   Log-Likelihood:            -6.5265e+05
No. Observations:               73804   AIC:                         1.305e+06
Df Residuals:                   73789   BIC:                         1.305e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Airline                   4.07

C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1781: RuntimeWarning: divide by zero encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
C:\Users\37789\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                  Feature       VIF
0                 Airline  1.130902
1                  Source  2.401026
2             Destination  3.372326
3     Total_Stopover_Time  2.029695
4               days_left  4.076351
5   travel_duration_hours  5.365978
6           num_stopovers  1.456109
7          departure_hour  1.158899
8            arrival_hour  1.231980
9             day_of_week  1.062061
10                  Month  3.988535
11                   Year  0.000000
12         Class_Business       inf
13          Class_Economy       inf
14            Class_First       inf
15  Class_Premium Economy       inf


In [22]:
X = X.drop(columns=['Class_First'])
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     4428.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:23:08   Log-Likelihood:            -6.5265e+05
No. Observations:               73804   AIC:                         1.305e+06
Df Residuals:                   73789   BIC:                         1.305e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Airline                   4.07

In [23]:
X = X.drop(columns=['Year'])
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.457
Model:                            OLS   Adj. R-squared:                  0.456
Method:                 Least Squares   F-statistic:                     4428.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:23:15   Log-Likelihood:            -6.5265e+05
No. Observations:               73804   AIC:                         1.305e+06
Df Residuals:                   73789   BIC:                         1.305e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  4518.53

In [24]:
X = X.drop(columns=['Class_Economy'])
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())

vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif_data)

                            OLS Regression Results                            
Dep. Variable:           price in CAD   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     4683.
Date:                Wed, 26 Jun 2024   Prob (F-statistic):               0.00
Time:                        16:23:20   Log-Likelihood:            -6.5295e+05
No. Observations:               73804   AIC:                         1.306e+06
Df Residuals:                   73790   BIC:                         1.306e+06
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  2815.45